In [1]:
# Import necessary libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime


def get_match_data(start_date, time_now_formatted, devengine):
    # Get historical match data on hard surface between start date and yesterday
    elo_hard = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Hard where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )

    # Get historical match data on clay surface between start date and yesterday
    elo_clay = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Clay where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )
    # Get historical match data on clay surface between start date and yesterday
    elo_grass = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_grass where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )
    # Get today's matches on hard surface that haven't yet been resulted
    elo_data_hard = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Hard where Date like '{time_now_formatted}' --and resulted like 'False'",
        con=devengine,
    )

    # Get today's matches on clay surface that haven't yet been resulted
    elo_data_clay = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Clay where Date like '{time_now_formatted}' --and resulted like 'False'",
        con=devengine,
    )

    # Get today's matches on clay surface that haven't yet been resulted
    elo_data_grass = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_grass where Date like '{time_now_formatted}' --and resulted like 'False'",
        con=devengine,
    )

    # Get historical match data on clay surface between start date and yesterday
    elo_grass = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_grass where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )    
    
    # Get historical match data on clay surface between start date and yesterday
    elo_all = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_All where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )

    # Get today's matches on clay surface that haven't yet been resulted
    elo_data_all = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_All where Date like '{time_now_formatted}' --and resulted like 'False'",
        con=devengine,
    )

    return elo_hard, elo_clay, elo_data_hard, elo_data_clay, elo_grass, elo_data_grass, elo_all, elo_data_all


# Connect to SQLite database using SQLAlchemy's create_engine
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
# Get current date and time
time_now = datetime.datetime.now()

# Format current date as string in YYYY-MM-DD format
time_now_formatted = time_now.strftime("%Y-%m-%d")

# Get the start date two years ago from today
today = time_now
two_years_ago = (today - datetime.timedelta(days=365 * 2)).strftime("%Y-%m-%d")

(
    elo_hard,
    elo_clay,
    elo_data_hard,
    elo_data_clay,
    elo_grass,
    elo_data_grass,
    elo_all,
    elo_data_all,    
) = get_match_data(two_years_ago, time_now_formatted, devengine)


def get_player_record(player, opponent_rank, history, range_low, range_high, auto):
    if auto:
        opponent_rank_low = opponent_rank - range_low
        opponent_rank_high = opponent_rank + range_high
    else:
        opponent_rank_low = range_low
        opponent_rank_high = range_high

    player_history = history[
        (
            (history["Fav"] == player)
            & (
                (history["Dog_Rank"] > opponent_rank_low)
                & (history["Dog_Rank"] < opponent_rank_high)
            )
        )
        | (
            (history["Dog"] == player)
            & (
                (history["Fav_Rank"] > opponent_rank_low)
                & (history["Fav_Rank"] < opponent_rank_high)
            )
        )
    ]
    if player_history.empty == False:
        result = float(
            len(player_history[player_history["Winner"] == player])
            / len(player_history)
        )
        return result, len(player_history)
    else:
        return 0, 0


def get_filtered_data(elo_data, elo):
    result_df = pd.DataFrame()
    for _, row in elo_data.sort_values(by="Time").iterrows():
        low_limit = 50
        high_limit = 50

        fav_percent, games = get_player_record(
            row.Fav, row.Dog_Rank, elo, low_limit, high_limit, True
        )
        count = 0
        while games < 10 and count < 200:
            count = count + 1
            low_limit = low_limit + 10
            high_limit = high_limit + 10
            fav_percent, games = get_player_record(
                row.Fav, row.Dog_Rank, elo, low_limit, high_limit, True
            )

        low_limit = 50
        high_limit = 50
        dog_percent, games2 = get_player_record(
            row.Dog, row.Fav_Rank, elo, low_limit, high_limit, True
        )
        count = 0
        while games2 < 10 and count < 200:
            count = count + 1
            low_limit = low_limit + 10
            high_limit = high_limit + 10
            dog_percent, games2 = get_player_record(
                row.Dog, row.Fav_Rank, elo, low_limit, high_limit, True
            )

        if games > 4 and games2 > 4:
            temp_df = pd.DataFrame(
                {
                    "Time": [row.Time],
                    "Fav_Odds": [row.Fav_Odds],
                    "Dog_Odds": [row.Dog_Odds],
                    "Fav": [row.Fav],
                    "Elo_Fav": [row.Elo_Fav],
                    "Fav_Record": ["{:.0%}".format(fav_percent)],
                    "Fav_Games": [games],
                    "Dog": [row.Dog],
                    "Dog_Odds": [row.Dog_Odds],
                    "Dog_Record": ["{:.0%}".format(dog_percent)],
                    "Dog_Games": [games2],
                    "fav_percent": [fav_percent],
                    "dog_percent": [dog_percent],
                    "Sex": [row.Sex],
                    "Resulted": [row.Resulted],
                    "fav_rank": [row.Fav_Rank],
                    "dog_rank": [row.Dog_Rank],
                    "Elo_Fav_Elo": [row.Elo_Fav_Elo],
                    "Elo_Dog_Elo": [row.Elo_Dog_Elo],
                }
            )
            result_df = pd.concat([result_df, temp_df])
    return result_df


results_hard = get_filtered_data(elo_data_hard, elo_hard)
results_clay = get_filtered_data(elo_data_clay, elo_clay)
results_grass = get_filtered_data(elo_data_grass, elo_grass)
#results_grass = get_filtered_data(elo_data_grass, elo_hard)
results_all = get_filtered_data(elo_data_all, elo_all)


def process_serving_data(result_df):
    try:
        # Try to read the 'servers_today.xlsx' file
        serving = pd.read_csv("mensserving.csv")

        # Try to read the 'servers_today_womens.xlsx' file
        serving_womens = pd.read_csv("womensserving.csv")

        # If 'serving_womens' dataframe is not empty, concatenate with 'serving' dataframe
        if serving_womens.empty == False:
            serving = pd.concat([serving, serving_womens])
        else:
            serving = serving

        # Drop the 'Time' column from the 'serving' dataframe
        # serving = serving.drop(columns='Time')
    except FileNotFoundError as e:
        # If either of the excel files is not found, print an error message and set serving to None
        print("The required excel file could not be found.")
        print("Error:", e)
        serving = None

    if serving is not None:
        try:
            # Try to merge the 'result_df' and 'serving' dataframes on the 'Fav' and 'Dog' columns
            result = pd.merge(
                result_df, serving, how="left", left_on=["Fav"], right_on=["Name"]
            )
            result = pd.merge(
                result, serving, how="left", left_on=["Dog"], right_on=["Name"]
            )
            result.rename(
                columns={
                    "Service Games Won_x": "Fav_Serve%",
                    "Service Games Won_y": "Dog_Serve%",
                    "Return Games Won_x": "Fav_Return%",
                    "Return Games Won_y": "Dog_Return%",
                },
                inplace=True,
            )
            result.drop(columns=["Name_x", "Name_y"], inplace=True)

            # Set 'final_hard' to the 'result' dataframe
            result_serving = result
        except Exception as e:
            # If an error occurs during merging, print an error message and set both 'result' and 'final_hard' to None
            print("Error occured while merging the dataframes.")
            print("Error:", e)
            result = None
            result_serving = None
    else:
        result = None
        result_serving = None

    return result, result_serving


_, serving_hard = process_serving_data(results_hard)
_, serving_clay = process_serving_data(results_clay)
_, serving_grass = process_serving_data(results_grass)
_, serving_all = process_serving_data(results_all)


data_concat = pd.DataFrame(columns=["Date", "Player", "Odds", "Win/Loss"])
for dataset_type in [("Winner", "Win"), ("Loser", "Loss")]:
    df = elo_hard[["Date", dataset_type[0], f"{dataset_type[0]}_Odds"]].copy()
    df["Player"] = df[dataset_type[0]]
    df["Odds"] = df[f"{dataset_type[0]}_Odds"]
    df["Win/Loss"] = dataset_type[1]
    df.drop(columns=[dataset_type[0], f"{dataset_type[0]}_Odds"], inplace=True)
    data_concat = pd.concat([data_concat, df])
data_concat = data_concat.sort_index()
data_concat["Odds"] = data_concat.Odds.astype(float)


def analyse_matchups(result_df, data_concat):
    for _, matchup in result_df.iterrows():
        player1 = matchup.Fav
        player2 = matchup.Dog
        player1_odds = float(matchup.Fav_Odds)
        player1_odds_hi = player1_odds + 0.15
        player1_odds_lo = player1_odds - 0.15
        player2_odds = float(matchup.Dog_Odds)
        player2_odds_hi = player2_odds + 0.15
        player2_odds_lo = player2_odds - 0.15
        player1 = data_concat[data_concat["Player"] == player1].copy()
        player2 = data_concat[data_concat["Player"] == player2].copy()
        player2 = player2[
            (player2["Odds"] > player2_odds_lo) & (player2["Odds"] < player2_odds_hi)
        ]
        if len(player2) > 0:
            winperc2 = len(player2[player2["Win/Loss"] == "Win"]) / len(player2)
        else:
            winperc2 = 0
        player1 = player1[
            (player1["Odds"] > player1_odds_lo) & (player1["Odds"] < player1_odds_hi)
        ]
        if len(player1) > 0:
            winperc1 = len(player1[player1["Win/Loss"] == "Win"]) / len(player1)
        else:
            winperc1 = 0
        if len(player1) > 5 and len(player2) > 3:
            print(
                matchup.Time,
                f"{matchup.Fav} ({round(player1_odds_lo,2)}-->{round(player1_odds_hi,2)})",
                f"{matchup.Dog} ({round(player2_odds_lo,2)}-->{round(player2_odds_hi,2)})",
            )
            print(len(player1), winperc1, len(player2), winperc2)


# analyse_matchups(serving_clay,data_concat)
final_hard = serving_hard
final_clay = serving_clay
final_grass = serving_grass
final_all = serving_all

In [2]:
def last_five(df,pastmatches):
    for index, row in df.iterrows():
        fav=row.Fav
        dog=row.Dog
        last_five_matches_fav=pastmatches[(pastmatches['Winner']==fav)|(pastmatches['Loser']==fav)].tail(5)
        if len(last_five_matches_fav)>0:  
            fav_last_five_win_perc=len(last_five_matches_fav[last_five_matches_fav['Winner']==fav])/len(last_five_matches_fav)
        else:
            fav_last_five_win_perc=0
        last_five_matches_dog=pastmatches[(pastmatches['Winner']==dog)|(pastmatches['Loser']==dog)].tail(5)
        if len(last_five_matches_dog)>0:    
            dog_last_five_win_perc=len(last_five_matches_dog[last_five_matches_dog['Winner']==dog])/len(last_five_matches_dog) 
        else:
            dog_last_five_win_perc=0
        df.at[index, 'fav_last_five_win_perc'] = fav_last_five_win_perc
        df.at[index, 'dog_last_five_win_perc'] = dog_last_five_win_perc
    return df

if final_hard is not None:
    final_hard=last_five(final_hard,elo_hard)
    final_hard['Fav_Odds']=final_hard['Fav_Odds'].astype(float)
    final_hard['Dog_Odds']=final_hard['Dog_Odds'].astype(float)    

In [3]:
if final_clay is not None:
    final_clay=last_five(final_clay,elo_clay)
    final_clay['Dog_Odds']=final_clay['Dog_Odds'].astype(float)
    final_clay['Fav_Odds']=final_clay['Fav_Odds'].astype(float)

In [4]:
if final_grass is not None:
    final_grass=last_five(final_grass,elo_grass)
    final_grass['Fav_Odds']=final_grass['Fav_Odds'].astype(float)
    final_grass['Dog_Odds']=final_grass['Dog_Odds'].astype(float)  
final_all=last_five(final_all,elo_all)
final_all['Fav_Odds']=final_all['Fav_Odds'].astype(float)
final_all['Dog_Odds']=final_all['Dog_Odds'].astype(float)        

In [5]:
for _,i in elo_data_hard.iterrows():
    check1=elo_hard[((elo_hard['Winner']==i.Winner)&(elo_hard['Loser']==i.Loser))|((elo_hard['Loser']==i.Winner)&(elo_hard['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

Tristan Schoolkate beat Omar Jasika


In [6]:
for _,i in elo_data_clay.iterrows():
    check1=elo_clay[((elo_clay['Winner']==i.Winner)&(elo_clay['Loser']==i.Loser))|((elo_clay['Loser']==i.Winner)&(elo_clay['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

Timofey Skatov beat Alejandro Tabilo


In [7]:
for _,i in elo_data_grass.iterrows():
    check1=elo_grass[((elo_grass['Winner']==i.Winner)&(elo_grass['Loser']==i.Loser))|((elo_grass['Loser']==i.Winner)&(elo_grass['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

Andy Murray beat Stefanos Tsitsipas


In [8]:
for _,i in elo_data_all.iterrows():
    check1=elo_all[((elo_grass['Winner']==i.Winner)&(elo_all['Loser']==i.Loser))|((elo_all['Loser']==i.Winner)&(elo_all['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

Sorana Cirstea beat Beatriz Haddad Maia
Andy Murray beat Stefanos Tsitsipas
Stefanos Tsitsipas beat Laslo Djere
Stefanos Tsitsipas beat Laslo Djere
Christopher Oconnell beat Christopher Eubanks


In [9]:
if final_hard is not None:
    final_hard=final_hard

else:
    final_hard=pd.DataFrame(columns=['Sex','Resulted','Time','Fav','Elo_Fav','Fav_Odds','fav_percent','fav_rank','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','dog_rank','Dog_Serve%','Dog_Return%','dog_last_five_win_perc'])

final_hard[(final_hard['Resulted']=='False')][['Time','Fav','Elo_Fav','Fav_Odds','fav_percent','fav_rank','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','dog_rank','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav,Elo_Fav,Fav_Odds,fav_percent,fav_rank,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,dog_rank,Dog_Serve%,Dog_Return%,dog_last_five_win_perc


In [10]:
if final_grass is not None:
    final_grass=final_grass

else:
    final_grass=pd.DataFrame(columns=['Sex','Resulted','Time','Fav','Elo_Fav','Fav_Odds','fav_percent','fav_rank','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','dog_rank','Dog_Serve%','Dog_Return%','dog_last_five_win_perc'])

final_grass[(final_grass['Sex']=='Mens')&(final_grass['Resulted']=='False')][['Time','Fav','Elo_Fav','Fav_Odds','fav_percent','fav_rank','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','dog_rank','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav,Elo_Fav,Fav_Odds,fav_percent,fav_rank,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,dog_rank,Dog_Serve%,Dog_Return%,dog_last_five_win_perc
11,20:00,Tommy Paul,Tommy Paul,1.84,0.600000,15.0,0.80,0.24,0.8,Jiri Lehecka,2.00,0.444444,37.0,0.75,0.18,0.4
12,21:30,Holger Rune,Holger Rune,1.36,0.625000,6.0,0.83,0.22,0.8,Alejandro Davidovich Fokina,3.24,0.375000,34.0,0.74,0.26,0.4
13,21:30,Christopher Eubanks,Christopher Eubanks,1.49,0.666667,43.0,0.87,0.11,1.0,Christopher Oconnell,2.68,0.500000,73.0,0.79,0.16,0.6
14,22:00,Daniil Medvedev,Daniil Medvedev,1.23,0.785714,3.0,0.86,0.27,0.8,Marton Fucsovics,4.31,0.700000,67.0,0.79,0.19,0.8
15,22:00,Stefanos Tsitsipas,Stefanos Tsitsipas,1.18,0.588235,5.0,0.86,0.21,0.4,Laslo Djere,4.98,0.444444,60.0,0.81,0.20,0.6


In [11]:
final_grass[(final_grass['Sex']=='Womens')&(final_grass['Resulted']=='False')][['Time','Fav','Elo_Fav','Fav_Odds','fav_percent','fav_rank','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','dog_rank','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav,Elo_Fav,Fav_Odds,fav_percent,fav_rank,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,dog_rank,Dog_Serve%,Dog_Return%,dog_last_five_win_perc
8,20:00,Beatriz Haddad Maia,Beatriz Haddad Maia,1.73,0.750000,13.0,0.72,0.31,0.6,Sorana Cirstea,2.14,0.500,37.0,0.65,0.38,0.6
9,20:00,Ekaterina Alexandrova,Ekaterina Alexandrova,1.18,0.857143,22.0,0.72,0.39,0.8,Dalma Galfi,4.98,0.400,126.0,0.67,0.32,0.4
10,20:00,Petra Kvitova,Petra Kvitova,1.07,1.000000,9.0,0.74,0.30,1.0,Natalija Stevanovic,9.02,0.875,225.0,0.00,0.00,1.0
16,22:00,Madison Keys,Madison Keys,1.31,0.875000,18.0,0.74,0.31,1.0,Marta Kostyuk,3.54,0.600,36.0,0.64,0.35,0.6


In [12]:
if final_clay is not None:
    final_clay=final_clay

else:
    final_clay=pd.DataFrame(columns=['Sex','Resulted','Time','Fav','Elo_Fav','Fav_Odds','fav_percent','fav_rank','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','dog_rank','Dog_Serve%','Dog_Return%','dog_last_five_win_perc'])

final_clay[(final_clay['Sex']=='Womens')&(final_clay['Resulted']=='False')][['Time','Fav','Elo_Fav','Fav_Odds','fav_percent','fav_rank','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','dog_rank','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav,Elo_Fav,Fav_Odds,fav_percent,fav_rank,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,dog_rank,Dog_Serve%,Dog_Return%,dog_last_five_win_perc


In [13]:
final_clay[(final_clay['Sex']=='Mens')&(final_clay['Resulted']=='False')][['Time','Fav','Elo_Fav','Fav_Odds','fav_percent','fav_rank','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','dog_rank','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav,Elo_Fav,Fav_Odds,fav_percent,fav_rank,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,dog_rank,Dog_Serve%,Dog_Return%,dog_last_five_win_perc
8,22:00,Alejandro Tabilo,Alejandro Tabilo,1.42,0.555556,145.0,0.72,0.29,0.8,Timofey Skatov,2.78,0.586207,138.0,0.71,0.18,0.4
9,22:30,Manuel Guinard,Manuel Guinard,1.83,0.846154,344.0,0.69,0.26,0.8,Duje Ajdukovic,1.93,0.363636,356.0,0.40,0.40,0.8
10,23:00,Facundo Diaz Acosta,Facundo Diaz Acosta,1.68,0.571429,115.0,0.00,0.00,0.8,Flavio Cobolli,2.13,0.277778,150.0,0.53,0.24,0.8
11,23:30,Giulio Zeppieri,Zsombor Piros,1.70,0.571429,125.0,0.79,0.15,0.8,Zsombor Piros,2.09,0.636364,113.0,0.00,0.00,0.6


In [14]:
if final_hard is not None:
    final_hard.to_pickle('Hard_Today')
else:
    final_clay[final_clay['Sex']=='k'].to_pickle('Hard_Today')
if final_grass is not None:
    final_grass.to_pickle('Grass_Today')
else:
    final_grass[final_grass['Sex']=='k'].to_pickle('Grass_Today')
if final_clay is not None:
    final_clay.to_pickle('Clay_Today')  
final_all.to_pickle('All_Today')    
final_clay=final_clay[(final_clay['Resulted']=='False')].copy()
final_hard=final_hard[(final_hard['Resulted']=='False')].copy()    

In [15]:
final_all[(final_all['Sex']=='Mens')&(final_all['Resulted']=='False')][['Time','Fav','Elo_Fav','Fav_Odds','fav_percent','fav_rank','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','dog_rank','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav,Elo_Fav,Fav_Odds,fav_percent,fav_rank,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,dog_rank,Dog_Serve%,Dog_Return%,dog_last_five_win_perc
28,20:00,Tommy Paul,Tommy Paul,1.84,0.491803,15.0,0.80,0.24,0.8,Jiri Lehecka,2.00,0.435897,37.0,0.75,0.18,0.4
31,21:30,Christopher Eubanks,Christopher Eubanks,1.49,0.583333,43.0,0.87,0.11,1.0,Christopher Oconnell,2.68,0.435897,73.0,0.79,0.16,0.6
32,21:30,Holger Rune,Holger Rune,1.36,0.641026,6.0,0.83,0.22,0.8,Alejandro Davidovich Fokina,3.24,0.367347,34.0,0.74,0.26,0.4
33,22:00,Stefanos Tsitsipas,Stefanos Tsitsipas,1.18,0.753086,5.0,0.86,0.21,0.4,Laslo Djere,4.98,0.233333,60.0,0.81,0.20,0.6
34,22:00,Daniil Medvedev,Daniil Medvedev,1.23,0.859155,3.0,0.86,0.27,0.8,Marton Fucsovics,4.31,0.371429,67.0,0.79,0.19,0.8
35,22:00,Alejandro Tabilo,Alejandro Tabilo,1.42,0.653846,145.0,0.72,0.29,0.6,Timofey Skatov,2.78,0.527778,138.0,0.71,0.18,0.6
37,22:30,Carlos Alcaraz,Carlos Alcaraz,1.08,0.820513,1.0,0.84,0.31,1.0,Nicolas Jarry,8.41,0.550000,28.0,0.87,0.05,0.6
38,22:30,Manuel Guinard,Duje Ajdukovic,1.83,0.789474,344.0,0.69,0.26,0.8,Duje Ajdukovic,1.93,0.400000,356.0,0.40,0.40,0.8
39,23:00,Facundo Diaz Acosta,Facundo Diaz Acosta,1.68,0.482759,115.0,0.00,0.00,0.6,Flavio Cobolli,2.13,0.307692,150.0,0.53,0.24,0.6
40,23:30,Giulio Zeppieri,Zsombor Piros,1.70,0.500000,125.0,0.79,0.15,0.6,Zsombor Piros,2.09,0.525000,113.0,0.00,0.00,0.8


In [16]:
final_all[(final_all['Sex']=='Womens')&(final_all['Resulted']=='False')][['Time','Fav','Elo_Fav','Fav_Odds','fav_percent','fav_rank','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','dog_rank','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav,Elo_Fav,Fav_Odds,fav_percent,fav_rank,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,dog_rank,Dog_Serve%,Dog_Return%,dog_last_five_win_perc
27,20:00,Beatriz Haddad Maia,Beatriz Haddad Maia,1.73,0.580645,13.0,0.72,0.31,0.6,Sorana Cirstea,2.14,0.452381,37.0,0.65,0.38,0.6
29,20:00,Petra Kvitova,Petra Kvitova,1.07,0.800000,9.0,0.74,0.30,1.0,Natalija Stevanovic,9.02,0.545455,225.0,0.00,0.00,1.0
30,20:00,Ekaterina Alexandrova,Ekaterina Alexandrova,1.18,0.764706,22.0,0.72,0.39,0.8,Dalma Galfi,4.98,0.333333,126.0,0.67,0.32,0.4
36,22:00,Madison Keys,Madison Keys,1.31,0.596154,18.0,0.74,0.31,1.0,Marta Kostyuk,3.54,0.422222,36.0,0.64,0.35,0.4
41,23:30,Anastasia Potapova,Anastasia Potapova,1.74,0.675000,23.0,0.63,0.42,0.8,Mirra Andreeva,2.13,0.700000,102.0,0.56,0.21,1.0
